# Post-processing Spectral Operations

In a post-processing task, computing derivatives is a common operation. Rayleigh computes derivatives using spectral transforms and recursion relations. This notebook describes a python
module that allows the computation of various spectral transforms and derivatives.

This notebook assumes that you are familiar with running Rayleigh and comfortable with manipulating the various types of diagnostic outputs using the provided python tools.

# I. Generate the Data

To fully understand this notebook, a particular model setup with specific outputs must be generated. Start by copying the *c2001_case0_minimal* input example (Boussinesq hydro benchmark) into a working directory and then rename it *main_input*.

## Change the radial grid
The first modification will be in the problemsize namelist. In the *main_input* file change the *problemsize_namelist* to be:
```
&problemsize_namelist
 l_max = 31
 domain_bounds = 0.53846153846153832d0, 1.04d0, 1.34d0, 1.5384615384615383d0
 ncheby = 24,16,14
 dealias_by = 6,4,2
/
```
This will build a radial domain composed of three sub-domains, each with a different resolution. The boundaries of the sub domains are explicitly set and the resulting domain should have an aspect ratio of 0.35 and a shell depth of 1.0, consistent with the chosen benchmark mode.

## Add some output quantities
The next modification will be to add specific output quantities. In the same *main_input* file change the *output_namelist* to be:
```
&output_namelist
 shellavg_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellavg_frequency = 2500
 shellavg_nrec = 10

 shellslice_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellslice_frequency = 2500
 shellslice_nrec = 10
 shellslice_levels_nrm = 0.5, 0.67, 0.8, 0.9

 shellspectra_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellspectra_frequency = 2500
 shellspectra_nrec = 10
 shellspectra_levels_nrm = 0.5, 0.67, 0.8, 0.9

 azavg_values = 1,2,3,10,19,28,11,20,29,12,21,30
 azavg_frequency = 2500
 azavg_nrec = 10
/
```
The added quantities correspond to:

| Quantity Code | Description |
|-------------- | ------------|
| 1             | $v_r$ |
| 2             | $v_\theta$ |
| 3             | $v_\phi$ |
| 10            | $\frac{\partial v_r}{\partial r}$ |
| 19            | $\frac{\partial v_r}{\partial \theta}$ |
| 28            | $\frac{\partial v_r}{\partial \phi}$ |
| 11            | $\frac{\partial v_\theta}{\partial r}$ |
| 20            | $\frac{\partial v_\theta}{\partial \theta}$ |
| 29            | $\frac{\partial v_\theta}{\partial \phi}$ |
| 12            | $\frac{\partial v_\phi}{\partial r}$ |
| 21            | $\frac{\partial v_\phi}{\partial \theta}$ |
| 30            | $\frac{\partial v_\phi}{\partial \phi}$ |

The final *main_input* file should look like this:
```
&problemsize_namelist
 l_max = 31
 domain_bounds = 0.53846153846153832d0, 1.04d0, 1.34d0, 1.5384615384615383d0
 ncheby = 24,16,14
 dealias_by = 6,4,2
/
&numerical_controls_namelist
/
&physical_controls_namelist
 benchmark_mode = 1
 benchmark_integration_interval = 100
 benchmark_report_interval = 5000
/
&temporal_controls_namelist
 max_iterations = 25000
 checkpoint_interval = 100000
 quicksave_interval = 10000
 num_quicksaves = 2
/
&io_controls_namelist
/
&output_namelist
 shellavg_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellavg_frequency = 2500
 shellavg_nrec = 10

 shellslice_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellslice_frequency = 2500
 shellslice_nrec = 10
 shellslice_levels_nrm = 0.5, 0.67, 0.8, 0.9

 shellspectra_values = 1,2,3,10,19,28,11,20,29,12,21,30
 shellspectra_frequency = 2500
 shellspectra_nrec = 10
 shellspectra_levels_nrm = 0.5, 0.67, 0.8, 0.9

 azavg_values = 1,2,3,10,19,28,11,20,29,12,21,30
 azavg_frequency = 2500
 azavg_nrec = 30
/

&Boundary_Conditions_Namelist
/
&Initial_Conditions_Namelist
/
&Test_Namelist
/
&Reference_Namelist
/
&Transport_Namelist
/
```

## Run Rayleigh to generate the code
Now that the *main_input* file has been properly configured, it is time to run Rayleigh and generate some data. The benchmark case should run to completion rather quickly. If a faster result is required, the *max_iterations* can be lowered, but be sure to change the *_frequency* and *_nrec* variables in each output quantity. 

**Remember:** the whole point of this exercise concerning the *main_input* file is to generate specific diagnostic quantities. The simulation does not need to run until completion; 25000 iterations was chosen because that is what the minimal input file uses, so there would be less editing of the input file.

# II. Extracting the Data
After generating the data, it must be read into python. You must ensure that the *rayleigh_diagnostics.py* file can be imported into python: either adjust your PYTHONPATH or copy the *rayleigh_diagnostics.py* file into your working directory.

The *spectral_utils.py* file will also be needed (it lives in the same directory as *rayleigh_diagnostics*.py), set your PYTHONPATH or copy this file to the working directory as well.

We will be reading AZ_Avgs, Shell_Avgs, Shell_Spectra, and Shell_Slices data.

In [ ]:
%matplotlib inline
from rayleigh_diagnostics import Shell_Avgs, AZ_Avgs, Shell_Slices, Shell_Spectra
import spectral_utils as SU
import matplotlib.pyplot as plt
import numpy as np
import os

If modifications were made to *max_iterations*, the *_frequency* values, or the *_nrec* values, then the following paths to the output files will need to be changed for your particular case.

In [ ]:
data_dir = "/home/orvedahl/cueball/Rayleigh-Runs/Testing/Spectral-Operations/"
shavg_itr  = "00025000"
azavg_itr  = "00025000"
shslc_itr  = "00025000"
shspec_itr = "00025000"

In [ ]:
shavg = Shell_Avgs(os.path.join(data_dir, "Shell_Avgs", shavg_itr), path='')
azavg = AZ_Avgs(os.path.join(data_dir, "AZ_Avgs", azavg_itr), path='')
shslc = Shell_Slices(os.path.join(data_dir, "Shell_Slices", shslc_itr), path='')
shspec = Shell_Spectra(os.path.join(data_dir, "Shell_Spectra", shspec_itr), path='')

Set some useful values for interfacing with the lookup table and extract some basic resolution data

In [ ]:
vel_inds = [1,2,3]
dr_inds  = [10,11,12]
dth_inds = [19,20,21]
dp_inds  = [28,29,30]

nth = shslc.ntheta
nphi = shslc.nphi

# III. Using the Spectral Objects
The *spectral_utils.py* file contains 4 classes:

1. *Fourier*
2. *Legendre*
3. *Chebyshev*
4. *SHT*

Each will be explored in turn.

## 1) Fourier
The *Fourier* class is designed to compute Fourier transforms of Rayleigh data. It also provides a method to compute derivatives with respect to phi/longitude.

It is initialized by supplying the resolution in the phi direction

In [ ]:
F = SU.Fourier(nphi)

Extract the velocity and theta/phi derivatives of the velocity from the Rayleigh data

In [ ]:
vel  = shslc.vals[:,:,:,shslc.lut[vel_inds],:] # shape (nphi,nth,nradii,3,ntime)
dvdt = shslc.vals[:,:,:,shslc.lut[dth_inds],:]
dvdp = shslc.vals[:,:,:,shslc.lut[ dp_inds],:]

Use the *Fourier* object to compute the derivative. The *d_dphi* method can compute the derivative of data that is already in spectral space: the *physical=True* argument specifies that the data is in physical space (the default).

In [ ]:
phi_deriv = F.d_dphi(vel, axis=0, physical=True)

Compute the error compared to the Rayleigh output

In [ ]:
error = np.abs(phi_deriv - dvdp)
print("Max d/dphi error = {}".format(np.max(error)))
print("Avg d/dphi error = {}".format(np.mean(error)))

The *Fourier* class can also compute forward and inverse Fourier transforms

In [ ]:
hybrid = F.to_spectral(vel, axis=0) # output will have shape (nfreq,nth,nradii,3,ntime)

phys = F.to_physical(hybrid, axis=0)

error = np.abs(vel - phys)
print("Max error = {}".format(np.max(error)))
print("Avg error = {}".format(np.mean(error)))

## 2) Legendre
The *Legendre* class is designed to compute Legendre transforms of Rayleigh data. It also provides a method to compute derivatives with respect to theta/co-latitude.

It is initialized by supplying the resolution in the theta direction

In [ ]:
L = SU.Legendre(nth)

Use the *Legendre* object to compute the derivative. The *d_dtheta* method can compute the derivative of data that is already in spectral space: the *physical=True* argument specifies that the data is in physical space (the default).

In [ ]:
#th_deriv = L.d_dtheta(vel, axis=1)
th_deriv = SU.ddx(vel, L.theta, axis=1)

Compute the error compared to the Rayleigh output

In [ ]:
error = np.abs(th_deriv - dvdt)
print("Max d/dth error = {}".format(np.max(error)))
print("Avg d/dth error = {}".format(np.mean(error)))

The *Legendre* class can also compute forward and inverse Legendre transforms

In [ ]:
hybrid = L.to_spectral(vel, axis=1)

phys = L.to_physical(hybrid, axis=1)

error = np.abs(vel - phys)
print("Max error = {}".format(np.max(error)))
print("Avg error = {}".format(np.mean(error)))